# Programmatic implementation of AmDock

Testing the programmatic htvs via the `AMDock` workflow. `AMDock` is a nive GUI for autodock that works very well. Below is a test run of my `htvs_amdock` module that tries to replicate the same workflow as `AMDock` programmatically.

In [ ]:
import importmonkey
import os

# set OMP_NUM_THREADS to number of available CPUs. Best for docking a single complex quickly
cpus = os.cpu_count() or 1
os.environ["OMP_NUM_THREADS"] = str(cpus)
home = os.environ["HOME"]

importmonkey.add_path(os.path.join(home, "gitrepos/gromacs_sims/scripts"))
from htvs_amdock import run_htvs

PROTEIN_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1hsg_mk1/experiment/1hsg_protein.pdb")
LIGAND_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1hsg_mk1/experiment/mk1.pdb")

testjob = [{
            "protein_name": "1hsg_protein",
            "ligand_name": "mk1",
            "protein_pdb": PROTEIN_PDB,
            "ligand_pdb": LIGAND_PDB,
            "working_dir": os.path.join(home, "gitrepos/gromacs_sims/htvs/1hsg_mk1/job_1hsg_mk1_script_20251206_propka"),
        }]

run_htvs(testjob)

## AutoLigand error (resolved)

======== AutoLigand Site Search (Automatic) FAILED STDOUT ========

======== AutoLigand Site Search (Automatic) FAILED STDERR ========
```python
Traceback (most recent call last):
  File "/usr/local/miniforge3/envs/htvs/bin/AutoLigand", line 7, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/miniforge3/envs/htvs/lib/python3.12/site-packages/AutoDockTools/AutoLigand.py", line 198, in main
    word = string.split(linee)
           ^^^^^^^^^^^^
AttributeError: module 'string' has no attribute 'split'
```

The `split()` function was deprecated and the functionality moved to a `xxx.split()` method during upgrade from python 2 to python 3. The `AutoLigand` script taken from [AutoDockTools_py3](https://github.com/Valdes-Tresanco-MS/AutoDockTools_py3) is supposed to translate the old `AutoLigand.py` written in python-2 to python-3, but fumbled it here. 


## Resolution
Raises this issue at the github repo, forked a copy of mine, then modded accordingly and it now works.

## Post-Processing

In [3]:
import os
home = os.environ["HOME"]
results_dir = os.path.join(home, "gitrepos/gromacs_sims/htvs/1hsg_mk1/job_1hsg_mk1_script_20251206_propka")
# Scan all log files in the results_dir and print a table of ligand names and their best binding affinities. Use tabulate module for formatting.
from tabulate import tabulate
import re
results = []
for filename in os.listdir(results_dir):
    if not filename.endswith(".log"):
        continue
    log_path = os.path.join(results_dir, filename)
    # extract pose id from filename (try common patterns, fallback to filename stem)
    m = re.search(r'pose[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'site[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'_(\d+)\.log$', filename)
    pose_id = m.group(1) if m else os.path.splitext(filename)[0]
    with open(log_path, "r") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if line.startswith("-----+"):
                if i + 1 < len(lines):
                    parts = lines[i+1].split()
                    if len(parts) >= 2:
                        try:
                            affinity = float(parts[1])
                        except ValueError:
                            continue
                        results.append((pose_id, affinity))

# sort by affinity in increasing numeric order and format for display
results.sort(key=lambda x: x[1])
table = [(pid, f"{aff:.3f}") for pid, aff in results]
print(tabulate(table, headers=["Pose ID", "Best Binding Affinity (kcal/mol)"]))

  Pose ID    Best Binding Affinity (kcal/mol)
---------  ----------------------------------
        6                             -11.1
        1                             -11.01
        2                             -10.99
        8                             -10.97
        9                              -7.123
       10                              -6.675
        5                              -6.476
        3                              -6.463
        4                              -6.361
        7                              -6.313


In [6]:
ligand_id = "mk1"
best_pose_id, best_affinity = results[0]
best_pose_ligand_pdb = os.path.join(results_dir, f"{ligand_id}_pose_site{best_pose_id}.pdb")
protein_pdb = testjob[0]["protein_pdb"]
#use nglview to visualize the best pose
import nglview as nv
view = nv.NGLWidget()
view.add_component(protein_pdb, ext="pdb", default_representation="cartoon", name="Protein")
view.add_component(best_pose_ligand_pdb, ext="pdb", default_representation="ball+stick", name="Best Ligand Pose")
view.center()
view.display()
view

NGLWidget()

## Unresolved Errors

The ligand PDB file refuses to protonate properly, either with openbabel or rdkit. `acpype` fails with odd number of electrons.
